In [210]:
import os
import shutil
import json
import pandas as pd
import subprocess
import time
import datetime
import itertools

In [167]:
protocol_file = './cfg.json'
BSS_CMD = '../border_security_system/target/debug/bss'

In [183]:
def make_problem_recipe(problem, path):
  assert 'field size' in problem
  assert 'num mobiles' in problem
  assert 'num statics' in problem

  fw = problem['field size'][0]
  fh = problem['field size'][1]
  nm = problem['num mobiles']
  ns = problem['num statics']

  obs_var = []
  if 'obstacles' in problem:
    with open(problem['obstacles']) as f:
      obs_var = json.load(f)

  recipe = json.dumps({
    "field_origin": [0.0, 0.0],
    "field_size": [fw, fh],
    "west_base_pos": [0.0, fh / 2],
    "east_base_pos": [fw, fh / 2],
    "num_mobiles": nm,
    "num_statics": ns,
    "num_obstacles": 0,
    "obstacle_variants": obs_var,
  })

  with open(path, mode='w') as f:
    f.write(recipe)

In [169]:
def parse_arg(cfg):
  return [{ 'key': cfg['key'], 'value': value } for value in cfg['values']]

In [170]:
def parse_args(cfg):
  arg_names = [ arg for arg in cfg ]
  expanded = [ parse_arg(cfg[arg]) for arg in cfg]
  traces = {}
  for arg_list in itertools.product(*expanded):
    trace = {}
    args = []
    for name, kv in zip(arg_names, arg_list):
      trace[name] = kv['value']
      args.append('{} {}'.format(kv['key'], kv['value']))
    args = ' '.join(args)
    traces[args] = trace
  return traces 


In [171]:
def parse_solver(cfg):
  return {
    'cmd': cfg['cmd'],
    'args': parse_args(cfg['args']),
  }

In [172]:
def parse_problem(cfg):
  expanded = [cfg[prop] for prop in cfg]
  prop_names = [ prop for prop in cfg]
  problems = []
  for props in itertools.product(*expanded):
    problems.append(dict(zip(prop_names, props)))
  return problems

In [265]:
def make_problem_instances(problem, working_dir, num):
  recipe_file = '{}/recipe.json'.format(working_dir)
  make_problem_recipe(problem, recipe_file)

  make_cmd = '{} make -r {} -d {} -n {}' \
    .format(BSS_CMD, recipe_file, working_dir, num)

  res = subprocess.call(make_cmd, shell=True)
  if res != 0:
    print('Error: %s' % make_cmd)
    exit(1)


def run_simulation(command, args, instance_file, working_dir):
  result_file = '%s/simulation_result.json' % working_dir
  if os.path.exists(result_file):
    os.remove(result_file)

  cmd = '%s %s %s -i %s -o %s' \
     % (BSS_CMD, command, args, instance_file, result_file)
  
  res = subprocess.call(cmd, shell=True)
  if res != 0:
    print('Error: %s' % cmd)
    exit(1)

  with open(result_file) as f:
    result = json.load(f)
    return result[-1]['laptime']


def expr(cfg):
  solvers = parse_solver(cfg['solver'])
  problems = parse_problem(cfg['problem'])
  trials = cfg['config']['trials']
  command = solvers['cmd']
  args_list = solvers['args']
  working_dir = cfg['config']['working dir']
  output_path = cfg['config']['output path']

  if os.path.exists(working_dir):
    print('Error: Specified working directory already exists: {}' \
      .format(working_dir))
    return

  if os.path.exists(output_path):
    print('Error: output file already exists: {}' \
      .format(output_path))
    return

  datetime_start = datetime.datetime.now()

  for i_pbm, problem in enumerate(problems):

    # Cleanup the working directly
    if os.path.exists(working_dir):
      shutil.rmtree(working_dir)
    os.mkdir(working_dir)

    # Generate problem instance files
    make_problem_instances(problem, working_dir, trials)

    for trial in range(trials):
      instance = '{}/instance-{}.json' \
        .format(working_dir, trial + 1)

      for i_args, args in enumerate(args_list):
        time_start = time.time()
        uptime = run_simulation(command, args, instance, working_dir)
        time_end = time.time()
        elapsed_time = time_end - time_start

        frame = {**args_list[args], **problem}
        frame['trial'] = '#{}'.format(trial)
        frame['uptime'] = uptime
        frame['elapsed time'] = elapsed_time
        frame = pd.DataFrame([frame.values()], columns=frame.keys())

        if os.path.exists(output_path):
          frame.to_csv(output_path, index=False, 
            encoding='utf-8', mode='a', header=False)
        else:
          frame.to_csv(output_path, index=False, 
            encoding='utf-8', mode='w')

        print('[{}] [problem #{}/{}] [instance #{}/{}] [solver #{}/{}] [elapsed-time: {:.1f} s] [uptime: {:.2f} h] {} {}'.format(
            datetime.datetime.now() - datetime_start, 
            i_pbm + 1, len(problems),
            trial + 1, trials,
            i_args + 1, len(args_list),
            elapsed_time,
            uptime / 3600,
            command, args))


In [266]:
with open('./cfg.json') as f:
  cfg = json.load(f)
expr(cfg)

[0:00:00.327583] [problem #1/18] [instance #1/1] [solver #1/2] [elapsed time: 0.3 s] [uptime: 185.59 h] single-bridge -a 0.0 -r 500
[0:00:00.691287] [problem #1/18] [instance #1/1] [solver #2/2] [elapsed time: 0.4 s] [uptime: 185.59 h] single-bridge -a 0.0 -r 1000
[0:00:00.917823] [problem #2/18] [instance #1/1] [solver #1/2] [elapsed time: 0.2 s] [uptime: 193.25 h] single-bridge -a 0.0 -r 500
[0:00:01.290105] [problem #2/18] [instance #1/1] [solver #2/2] [elapsed time: 0.4 s] [uptime: 184.61 h] single-bridge -a 0.0 -r 1000
[0:00:01.627572] [problem #3/18] [instance #1/1] [solver #1/2] [elapsed time: 0.3 s] [uptime: 186.46 h] single-bridge -a 0.0 -r 500
[0:00:01.784066] [problem #3/18] [instance #1/1] [solver #2/2] [elapsed time: 0.2 s] [uptime: 193.91 h] single-bridge -a 0.0 -r 1000
[0:00:02.792179] [problem #4/18] [instance #1/1] [solver #1/2] [elapsed time: 1.0 s] [uptime: 200.78 h] single-bridge -a 0.0 -r 500
[0:00:04.027836] [problem #4/18] [instance #1/1] [solver #2/2] [elapsed t

In [258]:
a = datetime.datetime.now()

In [259]:
b = datetime.datetime.now()

In [260]:
print(a, b)

2021-12-25 21:49:08.845100 2021-12-25 21:49:15.080556


In [264]:
str((b - a))

'0:00:06.235456'